In [3]:
import numpy as np

In [4]:
from core.genetic_fn import generate_population , fitness, tournament_selection, crossover, mutate, replace_firstweak

In [5]:
sample = [
    [0,2,1,2,1],
    [2,0,2,2,2],
    [1,2,0,2,1],
    [2,2,2,0,2],
    [1,2,1,2,0]
]

In [6]:
pop1 = generate_population(sample)
pop1

[[0, 1, 4, 3, 2],
 [3, 0, 1, 4, 2],
 [0, 3, 4, 1, 2],
 [2, 4, 0, 1, 3],
 [3, 0, 1, 4, 2],
 [0, 3, 1, 4, 2],
 [2, 1, 0, 4, 3],
 [4, 2, 1, 3, 0],
 [4, 2, 3, 0, 1],
 [1, 0, 4, 2, 3]]

In [7]:
fitness_value = fitness(pop1[0],sample)
fitness_value

0.1111111111111111

In [8]:
pop_fitness = [fitness(pop,sample) for pop in pop1]
pop_fitness

[0.1111111111111111,
 0.1111111111111111,
 0.1111111111111111,
 0.125,
 0.1111111111111111,
 0.125,
 0.1111111111111111,
 0.125,
 0.1111111111111111,
 0.125]

In [30]:
a = tournament_selection(pop1, pop_fitness, 4)
b = tournament_selection(pop1, pop_fitness, 4)
a[0] ,b[0]

([1, 0, 4, 2, 3], [0, 3, 1, 4, 2])

In [80]:
from typing import Optional
## THIS ONE VIOLTES THE CONSTRAINT OF THE TSP rules
def crossover(arg1, arg2, locus:Optional[int]=None):
    n = len(arg1)
    if locus is None:
        locus = np.random.randint(n)
    child1 = arg1[:locus] + arg2[locus:] 
    child2 = arg2[:locus] + arg1[locus:]
    assert (len(set(child1)) == n ) and (len(set(child2)) == n ) , "found duplicated node inside child"
    return child1, child2

def fixed_crossover(arg1, arg2):
    assert len(arg1) == len(arg2) , "parents' size should be equal"
    n = len(arg1)
    point = np.random.randint(n)
    child1 = arg2[:point]
    child2 = arg1[:point]
    return child1 + [x for x in arg1 if x not in child1] , child2 + [x for x in arg2 if x not in child2]

def fixed_crossover_twopoint(arg1, arg2):
    assert len(arg1) == len(arg2) , "parents' size should be equal"
    n = len(arg1)
    point1 = np.random.randint(n-1)
    point2 = np.random.choice(range(point1,n+1))
    child1 = arg2[point1:point2+1]
    child2 = arg1[point1:point2+1]
    return child1 + [x for x in arg1 if x not in child1] , child2 + [x for x in arg2 if x not in child2]
    
def ordered_crossover(arg1, arg2):
    assert len(arg1) == len(arg2) , "parents' size should be equal"
    n = len(arg1)
    child1 = [-1] * n
    child2 = [-1] * n

    start, end = np.sort(np.random.choice(n, 2, replace=False))
    
    child1[start:end+1] = arg2[start:end+1]
    child2[start:end+1] = arg1[start:end+1]

    remain1 = [x for x in arg1 if x not in child1]
    remain2 = [x for x in arg2 if x not in child2]

    child1 = [remain1.pop(0) if x == -1 else x for x in child1 ]
    child2 = [remain2.pop(0) if x == -1 else x for x in child2 ]

    return child1 , child2

def partialMap_crossover(arg1, arg2):
    n = len(arg1)
    # point1, point2 = np.sort(np.random.choice(n, 2, replace=False))
    point1 , point2 = 2,3
    child1 = [None] * n
    child2 = [None] * n

    child1[point1:point2+1] = arg2[point1:point2+1]
    child2[point1:point2+1] = arg1[point1:point2+1]

    for i in range(n):
        if i in list(range(point1,point2+1)):
            continue
        
        if arg1[i] not in child1:
            child1[i] = arg1[i]
    
        if arg2[i] not in child2:
            child2[i] = arg2[i]

    remain1 = [x for x in arg1 if x not in child1]
    remain2 = [x for x in arg2 if x not in child2]
    
    for i in range(n):
        if child1[i] == None:
            child1[i] = remain1.pop(0)
        if child2[i] == None:
            child2[i] = remain2.pop(0)

    return child1 , child2

In [81]:
partialMap_crossover(a[0], b[0])

([2, 0, 1, 4, 3], [0, 3, 4, 2, 1])

In [82]:
fixed_crossover(a[0],b[0])

([0, 3, 1, 4, 2], [1, 0, 4, 2, 3])

In [83]:
ordered_crossover(a[0],b[0])

([0, 3, 1, 4, 2], [1, 0, 4, 2, 3])

In [14]:
c, d = fixed_crossover(a[0], b[0])
c, d

([4, 0, 3, 1, 2], [0, 4, 2, 1, 3])

In [15]:
e = mutate(c)
f = mutate(d)
e, f

([1, 0, 3, 4, 2], [0, 4, 1, 2, 3])

In [16]:
pop1 = replace_firstweak(pop1, e, sample)
print(pop1)
pop1 = replace_firstweak(pop1,f, sample)
print(pop1)

[[0, 1, 4, 3, 2], [3, 0, 1, 4, 2], [0, 3, 4, 1, 2], [2, 4, 0, 1, 3], [3, 0, 1, 4, 2], [0, 3, 1, 4, 2], [2, 1, 0, 4, 3], [4, 2, 1, 3, 0], [4, 2, 3, 0, 1], [1, 0, 4, 2, 3]]
[[0, 1, 4, 3, 2], [3, 0, 1, 4, 2], [0, 3, 4, 1, 2], [2, 4, 0, 1, 3], [3, 0, 1, 4, 2], [0, 3, 1, 4, 2], [2, 1, 0, 4, 3], [4, 2, 1, 3, 0], [4, 2, 3, 0, 1], [1, 0, 4, 2, 3]]


In [17]:
population = generate_population(sample)
population_fitness = [fitness(pop,sample) for pop in population]

for i in range(10000):
    a = tournament_selection(population, population_fitness, 4)
    b = tournament_selection(population, population_fitness, 4)
    c, d = fixed_crossover(a[0], b[0])
    e = mutate(c)
    f = mutate(d)
    population = replace_firstweak(population, e, sample)
    population = replace_firstweak(population, f, sample)
    population_fitness = [fitness(pop,sample) for pop in pop1]

In [18]:
pop1, pop_fitness

([[0, 1, 4, 3, 2],
  [3, 0, 1, 4, 2],
  [0, 3, 4, 1, 2],
  [2, 4, 0, 1, 3],
  [3, 0, 1, 4, 2],
  [0, 3, 1, 4, 2],
  [2, 1, 0, 4, 3],
  [4, 2, 1, 3, 0],
  [4, 2, 3, 0, 1],
  [1, 0, 4, 2, 3]],
 [0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.125,
  0.1111111111111111,
  0.125,
  0.1111111111111111,
  0.125,
  0.1111111111111111,
  0.125])

In [19]:
[1/x for x in pop_fitness]

[9.0, 9.0, 9.0, 8.0, 9.0, 8.0, 9.0, 8.0, 9.0, 8.0]

In [20]:
import itertools

def calculate_total_distance(path, distance_matrix):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distance_matrix[path[i]][path[i + 1]]
    return total_distance

def tsp_bruteforce(distance_matrix):
    num_cities = len(distance_matrix)
    cities = list(range(num_cities))
    shortest_path = None
    shortest_distance = float('inf')

    for path in itertools.permutations(cities):
        distance = calculate_total_distance(path, distance_matrix)
        if distance < shortest_distance:
            shortest_distance = distance
            shortest_path = path

    return shortest_path, shortest_distance

# distance_matrix = [
#     [0, 2, 1, 2, 1],
#     [2, 0, 2, 2, 2],
#     [1, 2, 0, 2, 1],
#     [2, 2, 2, 0, 2],
#     [1, 2, 1, 2, 0]
# ]

best_path, best_distance = tsp_bruteforce(sample)
print("Best Path:", best_path)
print("Best Distance:", best_distance)


Best Path: (0, 2, 4, 1, 3)
Best Distance: 6


In [21]:
temp1 = [3,1,0,2,4]
temp2 = [0,2,4,3,1]


In [22]:
p1, p2 = 1, 2
n = len(temp1)

In [23]:
p1, p2 = 1, 2
n = len(temp1)
temp3 = [-1] * n
temp4 = [-1] * n 
temp3[p1:p2+1] = temp2[p1:p2+1]
temp4[p1:p2+1] = temp1[p1:p2+1]
remain1 = [x for x in temp1 if x not in temp3]
remain2 = [x for x in temp2 if x not in temp4]
result1 = [remain1.pop(0) if x == -1 else x for x in temp3 ]
result2 = [remain2.pop(0) if x == -1 else x for x in temp4 ]


In [24]:
remain1, remain2

([], [])

In [25]:
[remain1.pop(0) if x == -1 else x for x in temp3 ]

IndexError: pop from empty list

In [ ]:
[remain2.pop(0) if x == -1 else x for x in temp4 ]

[2, 1, 0, 4, 3]

In [ ]:
ordered_crossover(temp3, temp4)

IndexError: pop from empty list

In [ ]:
def ordered_crossover(parent1, parent2, p1, p2):
    n = len(parent1)
    child1 = [-1] * n
    child2 = [-1] * n
    
    child1[p1:p2+1] = parent2[p1:p2+1]
    child2[p1:p2+1] = parent1[p1:p2+1]
    
    remaining1 = [x for x in parent1 if x not in child1]
    remaining2 = [x for x in parent2 if x not in child2]
    
    child1 = [remaining1.pop(0) if x == -1 else x for x in child1]
    child2 = [remaining2.pop(0) if x == -1 else x for x in child2]
    
    return child1, child2


In [ ]:
ordered_crossover(temp1, temp2, 1, 2)

([3, 2, 4, 1, 0], [2, 1, 0, 4, 3])